### Demonstration on how to read a given section of the Gray data with the session class

The session class is part of the GDa (Gray Data Analysis) package, and it is used to read and save the session info of the gray data in h5 format for further processing such as the pairwise coherence computation and temporal network analysis.

It can be imported as:

In [1]:
%matplotlib inline
import sys
sys.path.insert(1, '/home/vinicius/GrayData-Analysis')

import h5py
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
from   wand.image        import Image as WImage # To plot pdf files in the notebook

from GDa.session         import session_info, session

pd.set_option("display.max_columns", None)

The session_info class will load the recording and trial information for a given session. These informations are stored on the files 'recording_info.mat' and 'trial_info.mat' and they contain the following information:

#### Recording info:
- image_locations: x,y location of images in degrees;
- image_names: names of the images;
- image_data: rgb matrices;
- channel_count: number of electrodes used during recording session;
- channel_numbers: electrode number;
- area: cell array containing the cortical area each electrode was in during recording session;
- depth: depth of each electrode during recording session;
- raw_data_sampling_rate: sampling rate of raw data
- calib_eye_sampling_rate: sampling rate of the eye data from Monkey Logic software
- number_sua: this is the number of single units on each electrode (note: all channels have mua);

#### Trial info:

- trial_type: 1 match-to-sample, 2 interleaved fixation trials, 3 blocked fixation trials, and 4 blank trials;
- sample_on: time that sample image was turned on (milliseconds);
- sample_off: time that sample image was turned off (milliseconds);
- match_on: time that the matching image and a distractor image are turned on (fixation point is simultaneously turned off);
- reaction_time: time taken for animal to make a saccade to the matching image;
- sample_image: number identifying which image was used for the sample;
- sample_location: number identifying the location of the sample image;
- match_image: number identifying the image that corresponds to the sample_image;
- match_location: number identifying the location of the matching image;
- nonmatch_image: number identifying the image that is not the matching image;
- nonmatch_location: number identifying the location of the non matching image;
- behavioral_response: 0 incorrect, 1 correct;
- num_trials: number of trials performed;

To instantiate a session_info class you have to call the constructor method and give the necessary parameters (go to the source code for further details):

In [2]:
info = session_info(raw_path = '../GrayLab/', monkey = 'lucy', date = '150128', session = 1)

This will load the inforamtion for monkey lucy, recording sesssion 1 and date 150128. Let's print the keys of the dictionaries to check the information that we just read:

In [3]:
print('Recording informations:\n')
print(info.recording_info.keys())

Recording informations:

dict_keys(['image_locations', 'image_names', 'image_data', 'channel_count', 'channel_numbers', 'area', 'depth', 'lfp_sampling_rate', 'calib_eye_sampling_rate', 'number_sua', 'ms_mod', 'slvr'])


In [4]:
info.trial_info.head()

,behavioral_response,match_image,match_location,match_on,nonmatch_image,nonmatch_location,num_trials,reaction_time,sample_image,sample_location,sample_off,sample_on,trial_type
0,NaN,NaN,NaN,2654.0,NaN,NaN,1006.0,NaN,NaN,NaN,1495.0,975.0,2.0
1,1.0,2.0,7.0,2824.0,4.0,3.0,1006.0,145.0,2.0,1.0,1371.0,851.0,1.0
2,1.0,5.0,8.0,2946.0,4.0,4.0,1006.0,205.0,5.0,1.0,1600.0,1093.0,1.0
3,1.0,2.0,3.0,2954.0,3.0,7.0,1006.0,260.0,2.0,1.0,1528.0,1021.0,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,1006.0,NaN,NaN,NaN,NaN,NaN,4.0


Trials can be easily selected using the __trial_info__ DataFrame, for instance if one wants to get the trials were a ODRT with correct behavioral response hapened, the following can be done:

In [5]:
filtered_trials = info.trial_info[(info.trial_info['trial_type']==1.0)\
                                   & (info.trial_info['behavioral_response']==1.0)]
# Print the data frane
filtered_trials.head()

,behavioral_response,match_image,match_location,match_on,nonmatch_image,nonmatch_location,num_trials,reaction_time,sample_image,sample_location,sample_off,sample_on,trial_type
1,1.0,2.0,7.0,2824.0,4.0,3.0,1006.0,145.0,2.0,1.0,1371.0,851.0,1.0
2,1.0,5.0,8.0,2946.0,4.0,4.0,1006.0,205.0,5.0,1.0,1600.0,1093.0,1.0
3,1.0,2.0,3.0,2954.0,3.0,7.0,1006.0,260.0,2.0,1.0,1528.0,1021.0,1.0
9,1.0,4.0,7.0,2759.0,2.0,3.0,1006.0,178.0,4.0,1.0,1480.0,973.0,1.0
12,1.0,2.0,9.0,2676.0,1.0,5.0,1006.0,161.0,2.0,1.0,1489.0,969.0,1.0


To load the data the class __session__ should be used. The class session inherits from session_info consequently its constuctor method receives the same parameters. However a few other parameters should be provided:

1. slvr_msmod (boolean, default = False): wheter to read or not data from channels with slvr and ms_mod;
2. align_to (str, default = 'cue'): wheter to align to 'cue' or 'match';
4. evt_dt (list, default = [-0.65, 3.00]): the begining and end time of the time-series relative to the reference (cue or match).
5. save_to_h5 (boolean, default = False): wheter to save to data to an h5 file;
6. h5_path (str, default = None): the path to save the h5 file;

In [6]:
# Instantiate class
s = session(raw_path = '../GrayLab/', monkey = 'lucy', date = '150128', session = 1,\
            slvr_msmod = False, align_to = 'cue', evt_dt = [-0.65, 3.00])

To load the data from the __.mat__ files, one can use the method __read_from_mat__. This will load all trial_types equals 1, 2, and 3 (1 match-to-sample, 2 interleaved fixation trials, 3 blocked fixation trials).

In [7]:
# Read data from .mat files
s.read_from_mat()

The data will be saved in an [xarray](http://xarray.pydata.org/en/stable/quick-overview.html#attributes) with three dimensions: trials, roi/channels, and time. The metadata is saved on the attributes dictionary.

In [8]:
print(s.data)

<xarray.DataArray (trials: 750, roi: 49, time: 3650)>
array([[[-2.93563432e-06, -2.64236696e-06, -3.83442998e-06, ...,
         -4.16106346e-06, -1.03178566e-05, -6.29233285e-06],
        [-1.85603334e-05, -8.58925699e-06,  3.90861701e-06, ...,
          1.72117696e-05, -2.42857766e-07, -1.14266870e-05],
        [-6.79570148e-05, -7.06648479e-05, -6.33662140e-05, ...,
          4.33440141e-05,  4.81205647e-05,  4.78378496e-05],
        ...,
        [ 1.98336570e-05,  2.35081457e-05,  2.56711124e-05, ...,
         -2.02410029e-04, -2.04568253e-04, -2.08600111e-04],
        [-5.83256139e-05, -5.48683857e-05, -4.65484587e-05, ...,
         -2.62835057e-04, -2.58350218e-04, -2.57071333e-04],
        [-1.41724684e-04, -1.51111225e-04, -1.54434589e-04, ...,
         -2.81779079e-04, -2.83306801e-04, -2.83594882e-04]],

       [[-1.96358590e-05, -3.32542587e-05, -3.89901593e-05, ...,
          2.48816053e-06,  1.02867565e-05,  1.44564632e-05],
        [-1.08752969e-05, -1.89363834e-05, -2.446

In [9]:
print('Session info.')
print('------------------------------------------------------------------')
print(s.data.attrs.keys())

Session info.
------------------------------------------------------------------
dict_keys(['nC', 'nP', 'pairs', 'fsample', 'channels_labels', 'stim', 'indch', 'areas', 't_cue_on', 't_cue_off', 't_match_on'])


The information stored in each of the keys in __readinfo__ is detailed bellow:

1. nC : Number of channels saved (not necessarily the total number of channels in the data);
2. nP : The total number of channel pairs.
3. nT : The total number of trials used.
4. pairs : A matrix $nP\times 2$, with the index of channels that form each pair.
5. indt : The number of each trial saved.
6. fsample : Frequency sample of the LFP signal.
7. tarray : Array containing the time axis (in our example it goes from -0.65 to 3.0).
8. channels_labels : Array containing the original channels labels.
9. stim : Stimulus label for each of the trials used.
10. indch: Index of the channels used.
11. areas: The name of the area of each channel.
12. t_cue_on: The time at which cue was turned on.
13. t_cue_off: The time at which cue was turned off.
14. t_match_on: The time at which match was turned on.

One can do all kind of filtering to the data, as aforementioned, the DataFrame __trial_info__ can be used to do so. Let's supose that one wants only the LFP data for coorect trials of the type 1 (ODRT). 

In [10]:
idx = s.trial_info[(s.trial_info['trial_type']==1.0) \
                    & (s.trial_info['behavioral_response']==1.0)].index.values

In [11]:
data_filtered = s.data.sel(trials=idx)